Useful code
https://www.kaggle.com/papadopc/sf-crime/neural-nets-and-address-featurization/code

In [20]:
import pandas as pd
import os
import numpy as np
from sklearn.cross_validation import KFold
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.metrics import log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import ExtraTreesClassifier

train_data = pd.read_csv('train.csv', parse_dates =['Dates'], dtype='object', index_col=False)
test_data = pd.read_csv('test.csv', parse_dates =['Dates'], dtype='object', index_col=0)

In [21]:
train_data[:2]

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747


In [22]:
test_data[:2]

,Dates,DayOfWeek,PdDistrict,Address,X,Y
Id,,,,,,
0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.39958770418998,37.7350510103906
1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391522893042,37.7324323864471


In [23]:
pd.value_counts(train_data["Category"])

LARCENY/THEFT                  174900
OTHER OFFENSES                 126182
NON-CRIMINAL                    92304
ASSAULT                         76876
DRUG/NARCOTIC                   53971
VEHICLE THEFT                   53781
VANDALISM                       44725
WARRANTS                        42214
BURGLARY                        36755
SUSPICIOUS OCC                  31414
MISSING PERSON                  25989
ROBBERY                         23000
FRAUD                           16679
FORGERY/COUNTERFEITING          10609
SECONDARY CODES                  9985
WEAPON LAWS                      8555
PROSTITUTION                     7484
TRESPASS                         7326
STOLEN PROPERTY                  4540
SEX OFFENSES FORCIBLE            4388
DISORDERLY CONDUCT               4320
DRUNKENNESS                      4280
RECOVERED VEHICLE                3138
KIDNAPPING                       2341
DRIVING UNDER THE INFLUENCE      2268
RUNAWAY                          1946
LIQUOR LAWS 

In [24]:
df1 = train_data[train_data.Category == "TREA"]
df2 = train_data[train_data.Category == "PORNOGRAPHY/OBSCENE MAT"]
df3 = train_data[train_data.Category == "GAMBLING"]
df4 = train_data[train_data.Category == "SEX OFFENSES NON FORCIBLE"]
df5 = train_data[train_data.Category == "EXTORTION"]
df6 = train_data[train_data.Category == "BRIBERY"]
df7 = train_data[train_data.Category == "BAD CHECKS"]
df8 = train_data[train_data.Category == "FAMILY OFFENSES"]
df9 = train_data[train_data.Category == "SUICIDE"]
df10 = train_data[train_data.Category == "EMBEZZLEMENT"]

train_data = pd.concat([train_data, df1, df1, df1, df1, df1, df1, df1, df1, df1, df1, df1, df1, df1, df1, df1, df1, 
                        df2, df2, df2, df2,
                        df3, df3,
                        df4, df4,
                        df5,df6,df7,df8,df9,df10])

pd.value_counts(train_data["Category"])

LARCENY/THEFT                  174900
OTHER OFFENSES                 126182
NON-CRIMINAL                    92304
ASSAULT                         76876
DRUG/NARCOTIC                   53971
VEHICLE THEFT                   53781
VANDALISM                       44725
WARRANTS                        42214
BURGLARY                        36755
SUSPICIOUS OCC                  31414
MISSING PERSON                  25989
ROBBERY                         23000
FRAUD                           16679
FORGERY/COUNTERFEITING          10609
SECONDARY CODES                  9985
WEAPON LAWS                      8555
PROSTITUTION                     7484
TRESPASS                         7326
STOLEN PROPERTY                  4540
SEX OFFENSES FORCIBLE            4388
DISORDERLY CONDUCT               4320
DRUNKENNESS                      4280
RECOVERED VEHICLE                3138
KIDNAPPING                       2341
EMBEZZLEMENT                     2332
DRIVING UNDER THE INFLUENCE      2268
RUNAWAY     

In [25]:
data = train_data
print(train_data.count())

Dates         881937
Category      881937
Descript      881937
DayOfWeek     881937
PdDistrict    881937
Resolution    881937
Address       881937
X             881937
Y             881937
dtype: int64


In [26]:
categories = pd.unique(data['Category'])
daysOfWeek = pd.unique(data['DayOfWeek'])
pdDistricts = pd.unique(data['PdDistrict'])

print("categories:", len(categories), "\npdDistricts:", len(pdDistricts))

for i in range(len(categories)):
    data.loc[data["Category"] == categories[i], "Category"] = i
data["Category"] = data["Category"].astype(float)
for i in range(len(daysOfWeek)):
    data.loc[data["DayOfWeek"] == daysOfWeek[i], "DayOfWeek"] = i
data["DayOfWeek"] = data["DayOfWeek"].astype(float)
for i in range(len(pdDistricts)):
    data.loc[data["PdDistrict"] == pdDistricts[i], "PdDistrict"] = i
data["PdDistrict"] = data["PdDistrict"].astype(float)

data["X"] = data["X"].astype(float)
data["Y"] = data["Y"].astype(float)

data.dtypes

('categories:', 39, '\npdDistricts:', 10)


Dates         datetime64[ns]
Category             float64
Descript              object
DayOfWeek            float64
PdDistrict           float64
Resolution            object
Address               object
X                    float64
Y                    float64
dtype: object

In [27]:
# make PdDistrict a dummy variable


df = pd.get_dummies(data[["DayOfWeek","PdDistrict", "Category"]], columns=["PdDistrict"])
df[:2]

,DayOfWeek,Category,PdDistrict_0.0,PdDistrict_1.0,PdDistrict_2.0,PdDistrict_3.0,PdDistrict_4.0,PdDistrict_5.0,PdDistrict_6.0,PdDistrict_7.0,PdDistrict_8.0,PdDistrict_9.0
0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0


In [30]:
from sklearn.ensemble import ExtraTreesClassifier


predictors = ["DayOfWeek", 'PdDistrict_0.0', 'PdDistrict_1.0', 'PdDistrict_2.0', 'PdDistrict_3.0', 'PdDistrict_4.0',
              'PdDistrict_5.0', 'PdDistrict_6.0', 'PdDistrict_7.0', 'PdDistrict_8.0', 'PdDistrict_9.0']

features, labels = df[predictors], df["Category"]
features.index=range(len(features))
labels.index=range(len(labels))
#scaler = StandardScaler()
#X = scaler.fit_transform(X)


clf = ExtraTreesClassifier(n_estimators=9, max_depth=None, min_samples_split=1, random_state=0)
sss = StratifiedShuffleSplit(labels, n_iter=5, test_size=0.2, train_size=0.2, random_state=42)
   
for train_index, test_index in sss:
    features_train,features_test=features.iloc[train_index],features.iloc[test_index]
    labels_train,labels_test=labels[train_index],labels[test_index]
features_test.index=range(len(features_test))
features_train.index=range(len(features_train))
labels_train.index=range(len(labels_train))
labels_test.index=range(len(labels_test))
features.index=range(len(features))
labels.index=range(len(labels))

#model = LogisticRegression()
model = ExtraTreesClassifier(n_estimators=9, max_depth=None, min_samples_split=1, random_state=0)
model.fit(features_train,labels_train)

print("all", log_loss(labels, model.predict_proba(features.as_matrix())))
print("train", log_loss(labels_train, model.predict_proba(features_train.as_matrix())))
print("test", log_loss(labels_test, model.predict_proba(features_test.as_matrix())))

('all', 2.6562574657211377)
('train', 2.6238775773445102)
('test', 2.6664874471311735)


In [23]:
districts_columns=df.columns[8:]
predictors = districts_columns.append("DayOfWeek")

X, y = df[predictors], df["Category"]
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 4 classifieres:
clf1 = RandomForestClassifier(n_estimators=25)
clf2 = RandomForestClassifier(n_estimators=25)
clf3 = RandomForestClassifier(n_estimators=25)
clf4 = RandomForestClassifier(n_estimators=25)
clfs = [clf1, clf2, clf3, clf4, clf5]

sss = StratifiedShuffleSplit(y, n_iter=5, test_size=0.2, random_state=42)

for train_index, test_index in sss:
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_80, X_20 = X[train_index], X[test_index]
    y_80, y_20 = y[train_index], y[test_index]
    
    for clf_i in clfs:
        sss1 = StratifiedShuffleSplit(y_80[y_train], n_iter=2, train_size=0.2, test_size=0.2 random_state=42)
        for train_index1, test_index1 in sss1:
            clf_i.fit(X_80[train_index1], y_80[train_index1])
            clf_probs = clf_i.predict_proba(X_80[test_index1])
            score = log_loss(y_80[test_index1], clf_probs)
    
    # need to make bagging of clfs
    # clfs = ...


    clfs.fit(X_train, y_train)
    clfs_probs = clfs.predict_proba(X_20)
    
    sig_clf = CalibratedClassifierCV(clfs, method="sigmoid", cv="prefit")
    
    sig_clf.fit(X_valid, y_valid)
    sig_clf_probs = sig_clf.predict_proba(X_test)
    sig_score = log_loss(y_test, sig_clf_probs)



SyntaxError: invalid syntax (<ipython-input-23-a50017266649>, line 23)

Other approaches

In [ ]:
#Here I was trying to make SVM + grid search

districts_columns=df.columns[8:]
predictors = ["DayOfWeek",'PdDistrict_1.0', 'PdDistrict_2.0', 'PdDistrict_3.0', 'PdDistrict_4.0', 'PdDistrict_5.0',
              'PdDistrict_6.0', 'PdDistrict_7.0', 'PdDistrict_8.0', 'PdDistrict_9.0']

X, y = df[predictors], df["Category"]
scaler = StandardScaler()
X = scaler.fit_transform(X)

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(y, n_iter=2, test_size=0.05, train_size=0.2, random_state=42)

grid = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=cv)
grid.fit(X, y)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

In [65]:
# Train uncalibrated random forest classifier on whole train and validation
# data and evaluate on test data
clf = RandomForestClassifier(n_estimators=25)
clf.fit(X_train_valid, y_train_valid)
clf_probs = clf.predict_proba(X_test)
score = log_loss(y_test, clf_probs)


# Train random forest classifier, calibrate on validation data and evaluate
# on test data
clf = RandomForestClassifier(n_estimators=25)
clf.fit(X_train, y_train)
clf_probs = clf.predict_proba(X_test)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid", cv="prefit")
sig_clf.fit(X_valid, y_valid)
sig_clf_probs = sig_clf.predict_proba(X_test)
sig_score = log_loss(y_test, sig_clf_probs)

ValueError: y_true and y_pred have different number of classes 31, 35

In [53]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.4, random_state=0)
X_train.shape, y_train.shape
X_test.shape, y_test.shape

alg = RandomForestClassifier(random_state=1, n_estimators=10, min_samples_split=2, min_samples_leaf=1)

scores = cross_validation.cross_val_score(alg, X, y, cv=6)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf2 = SGDClassifier(loss="hinge", penalty="l2")

scores = cross_validation.cross_val_score(clf2, X, y, cv=6)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.27 (+/- 0.01)


In [17]:
# The predictions are in three separate numpy arrays.  Concatenate them into one.  
# We concatenate them on axis 0, as they only have one axis.
predictions = np.concatenate(predictions, axis=0)

# Map predictions to outcomes (only possible outcomes are 1 and 0)
predictions[predictions > .5] = 1
predictions[predictions <=.5] = 0

accuracy = sum([1 if predictions[i]==data["Category"][i] else 0 for i in range(len(predictions))]) / len(data["Category"])

print(accuracy)

0


In [ ]:
#For submission

# Initialize the algorithm class
alg = LogisticRegression(random_state=1)

# Train the algorithm using all the training data
alg.fit(titanic[predictors], titanic["Survived"])

# Make predictions using the test set.
predictions = alg.predict(titanic_test[predictors])

# Create a new dataframe with only the columns Kaggle wants from the dataset.
submission = pandas.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })